In [3]:
import numpy as np

## DES

In [4]:
# HELPER FUNCTIONS

def arr_to_int(binary_array: np.array) -> int:
    """Converts np.array with bit values to int"""
    result = 0
    for b in binary_array:
        result = (result<<1) + b
    return result

def int_to_arr(n: int, out_length=8) -> np.array:
    """Converts int to np.array with bit values"""
    result = []
    for _ in range(out_length):
        result += [n%2]
        n//=2
    return np.array(result[::-1])

In [5]:
# EXAMPLE
n = 123
arr = int_to_arr(n)
back_to_n = arr_to_int(arr)

print(n, arr, back_to_n,  sep="\n")

123
[0 1 1 1 1 0 1 1]
123


In [6]:
# HELPER FUNCTIONS 

def msg_to_bits(msg: str) -> np.array:
    msg = [char for char in bytearray(msg, 'ascii')]
    msg = [int_to_arr(char, out_length=8) for char in msg]
    msg = np.concatenate(msg)
    msg 
    return msg

def bits_to_msg(msg: np.array) -> str:
    msg = [msg[i:i+8] for i in range(0, len(msg), 8)]
    msg = [arr_to_int(arr) for arr in msg]
    msg = bytearray(msg).decode('ascii')
    return msg

In [7]:
# EXAMPLE
msg = "hej"
arr = msg_to_bits(msg)
back_to_msg = bits_to_msg(arr)

print(msg, arr, back_to_msg, sep="\n")

hej
[0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 0 1 0]
hej


### Key Schedule

In [8]:
PC1 = [56, 48, 40, 32, 24, 16, 8, 0, 57, 49, 41, 33, 25, 17, 9, 1, 58, 50, 42, 34, 26, 18, 10, 2, 59, 51, 43, 35, 62, 54, 46, 38, 30, 22, 14, 6, 61, 53, 45, 37, 29, 21, 13, 5, 60, 52, 44, 36, 28, 20, 12, 4, 27, 19, 11, 3]
PC2 = [13, 16, 10, 23, 0, 4, 2, 27, 14, 5, 20, 9, 22, 18, 11, 3, 25, 7, 15, 6, 26, 19, 12, 1, 40, 51, 30, 36, 46, 54, 29, 39, 50, 44, 32, 47, 43, 48, 38, 55, 33, 52, 45, 41, 49, 35, 28, 31]
round_shifts = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

def key_schedule(key: np.array, reversed=False):
    result = []
    key = key[PC1]   # initial permutation+compression
    for shift in round_shifts:
        left, right = key[:len(key)//2], key[len(key)//2:]   # split
        left, right = np.roll(left, shift), np.roll(right, shift)  # circular roll
        key = np.concatenate([left, right])  # concatenate
        result.append(key[PC2]) # round key after final permutation
    return result if not reversed else result[::-1]

### SBOX nonlinearity

In [9]:
SBOX_TABLE = [
[[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],[0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],[4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],[15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]],
[[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],[3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],[0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],[13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]],
[[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],[13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],[13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],[1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]],
[[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],[13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],[10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],[3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]],
[[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],[14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],[4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],[11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]],
[[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],[10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],[9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],[4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]],
[[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],[13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],[1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],[6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]],
[[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],[1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],[7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],[2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]]
]

def _sbox(block_part, i):
    sbox = SBOX_TABLE[i]
    first_last = arr_to_int(block_part[[0,5]])
    middle = arr_to_int(block_part[1:5])
    sbox_out = sbox[first_last][middle]
    return int_to_arr(sbox_out, out_length=4)

def sbox_substitute(block):
    block_parts = [block[i:i+6] for i in range(0, len(block), 6)]
    block_parts = [_sbox(part, i) for i, part in enumerate(block_parts)]
    return np.concatenate(block_parts)

### Round Function

In [10]:
EXP = [31, 0, 1, 2, 3, 4, 3, 4, 5, 6, 7, 8, 7, 8, 9, 10, 11, 12, 11, 12, 13, 14, 15, 16, 15, 16, 17, 18, 19, 20, 19, 20, 21, 22, 23, 24, 23, 24, 25, 26, 27, 28, 27, 28, 29, 30, 31, 0]
PER = [15, 6, 19, 20, 28, 11, 27, 16, 0, 14, 22, 25, 4, 17, 30, 9, 1, 7, 23, 13, 31, 26, 2, 8, 18, 12, 29, 5, 21, 10, 3, 24]

def F(block: np.array, round_key: np.array):
    block = block[EXP]  # block expansion
    block = block ^ round_key  # xoring with round key
    block = sbox_substitute(block)  # apply nonlinearity
    block = block[PER]  # final permutation
    return block


### Cipher and Decipher operations

In [11]:
P = [57, 49, 41, 33, 25, 17, 9, 1, 59, 51, 43, 35, 27, 19, 11, 3, 61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39, 31, 23, 15, 7, 56, 48, 40, 32, 24, 16, 8, 0, 58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36, 28, 20, 12, 4, 62, 54, 46, 38, 30, 22, 14, 6]
INV_P = [39, 7, 47, 15, 55, 23, 63, 31, 38, 6, 46, 14, 54, 22, 62, 30, 37, 5, 45, 13, 53, 21, 61, 29, 36, 4, 44, 12, 52, 20, 60, 28, 35, 3, 43, 11, 51, 19, 59, 27, 34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9, 49, 17, 57, 25, 32, 0, 40, 8, 48, 16, 56, 24]


def cipher(block, key):
    block = block[P]
    for round_key in key_schedule(key):
        L, R = np.split(block, 2)
        L = L ^ F(R, round_key)
        block = np.concatenate([R, L])
    block = block[INV_P]
    
    return block


def decipher(block, key):
    block = block[P]
    for round_key in key_schedule(key, reversed=True):
        L, R = np.split(block, 2)
        R = R ^ F(L, round_key)
        block = np.concatenate([R, L])
    block = block[INV_P]
    
    return block
        

In [12]:
plaintext = msg_to_bits("siemanko")
key = msg_to_bits("12345678")

ciphertext = cipher(plaintext, key)
deciphered = decipher(ciphertext, key)


print("-"*20)
print("Plaintext:")
print(bits_to_msg(plaintext))
print(plaintext)

print("-"*20)
print("Key:")
print(key)

print("-"*20)

print("Ciphertext:")
print(ciphertext)

print("-"*20)

print("Deciphered:")
print(bits_to_msg(deciphered))
print(deciphered)
print("-"*20)

--------------------
Plaintext:
siemanko
[0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0
 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1]
--------------------
Key:
[0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 0
 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 1 0 0 0]
--------------------
Ciphertext:
[1 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1
 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1]
--------------------
Deciphered:
siemanko
[0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0
 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1]
--------------------


In [13]:
ciphertext.mean()

0.515625